In [1]:
import os
import pandapower as pp
import polars as pl
from polars import col as c

from pipelines.dig_a_plan_d_model import DigAPlan
from data_display.grid_plotting import plot_grid_from_pandapower
from data_display.output_processing import compare_dig_a_plan_with_pandapower
from data_connector import change_schema_to_dig_a_plan_schema, duckdb_to_changes_schema
from twindigrid_changes.schema import ChangesSchema
from networkx_function import generate_nx_edge, get_connected_edges_data, generate_bfs_tree_with_edge_data, get_all_edge_data
import networkx as nx

import matplotlib.pyplot as plt

os.chdir(os.getcwd().replace("/src", ""))
os.environ['GRB_LICENSE_FILE'] = os.environ["HOME"] + "/gurobi_license/gurobi.lic"


In [2]:
LOAD_FACTOR  = 1

# set input data

net = pp.from_pickle(".cache/input_data/mv_example.p")

net["load"]["p_mw"] = net["load"]["p_mw"]*LOAD_FACTOR
net["load"]["q_mvar"] = net["load"]["q_mvar"]*LOAD_FACTOR
net["trafo"]["vn_hv_kv"] = [110., 110.]
net["trafo"]["vn_lv_kv"] = [20, 17.]

line_list= [6, 9, 11, 13, 16]


net["line"].loc[10, "max_i_ka"] = 40e-3
# net["line"].loc[line_list, "max_i_ka"] = 1e-3

In [3]:
change_schema: ChangesSchema = duckdb_to_changes_schema(file_path=".cache/input_data/boisy_grid.db")
# 2) build the *original* schema once

base_grid_data = change_schema_to_dig_a_plan_schema(change_schema = change_schema)

nx_graph = nx.Graph()
_ = base_grid_data["edge_data"].with_columns(
    pl.struct(pl.all()).pipe(generate_nx_edge, nx_graph=nx_graph)
)

edge_id  = get_connected_edges_data(nx_graph = nx_graph).filter(c("graph_id")!= 0)["edge_id"].to_list()
node_id = get_connected_edges_data(nx_graph = nx_graph).filter(c("graph_id")!= 0)\
    .unpivot(on=["u_of_edge", "v_of_edge"])["value"].to_list()
base_grid_data["edge_data"] = base_grid_data["edge_data"].filter(~c("edge_id").is_in(edge_id))
base_grid_data["node_data"] = base_grid_data["node_data"].filter(~c("node_id").is_in(node_id))
base_grid_data["edge_data"] = base_grid_data["edge_data"].with_columns(pl.when(c("b_pu") < 1e-6).then(pl.lit(0)).otherwise(c("b_pu")).alias("b_pu"))


Read and validate tables from boisy_grid.db file: 100%|██████████████████████████████████████████████████████████████| 28/28 [00:00<00:00, 312.96it/s]


In [4]:

dig_a_plan: DigAPlan = DigAPlan(
    verbose= True, big_m = 1e4, penalty_cost = 1e1
)

dig_a_plan.add_grid_data(**base_grid_data)
dig_a_plan.solve_models_pipeline(max_iters = 200)

Master obj: 0, Slave obj: 0 and Gap: 1e6:   0%|          | 0/200 [00:00<?, ?it/s]

Read LP format model from file /tmp/tmp9sgtspsf.pyomo.lp
Reading time = 0.03 seconds
x1: 13091 rows, 11428 columns, 38940 nonzeros
Set parameter QCPDual to value 1
Set parameter NonConvex to value 2
Set parameter QCPDual to value 1
Set parameter NumericFocus to value 1
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Ubuntu 22.04.5 LTS")

2025-06-23 15:00:17 B01BPC15 gurobipy[128998] INFO Read LP format model from file /tmp/tmp9sgtspsf.pyomo.lp
2025-06-23 15:00:17 B01BPC15 gurobipy[128998] INFO Reading time = 0.03 seconds
2025-06-23 15:00:17 B01BPC15 gurobipy[128998] INFO x1: 13091 rows, 11428 columns, 38940 nonzeros
2025-06-23 15:00:17 B01BPC15 gurobipy[128998] INFO Set parameter QCPDual to value 1
2025-06-23 15:00:17 B01BPC15 gurobipy[128998] INFO Set parameter NonConvex to value 2
2025-06-23 15:00:17 B01BPC15 gurobipy[128998] INFO Set parameter QCPDual to value 1
2025-06-23 15:00:17 B01BPC15 gurobipy[128998] INFO Set parameter NumericFocus to value 1
2025-06-23 15:00:17 

Master obj: -1.0E-08, Slave obj: 3.4132E+02 and Gap: INF:   0%|          | 0/200 [00:00<?, ?it/s]

shape: (2_104, 4)
┌──────────────┬───────────────┬─────┬─────────────────┐
│ LC           ┆ marginal_cost ┆ d   ┆ d_variable      │
│ ---          ┆ ---           ┆ --- ┆ ---             │
│ str          ┆ f64           ┆ i64 ┆ object          │
╞══════════════╪═══════════════╪═════╪═════════════════╡
│ 0,36,0       ┆ 774.843438    ┆ 1   ┆ d[0,36,0]       │
│ 1,1,108      ┆ 410.699376    ┆ 1   ┆ d[1,1,108]      │
│ 2,4,19       ┆ -5.4041e-11   ┆ 0   ┆ d[2,4,19]       │
│ 3,27,5       ┆ -5.4041e-11   ┆ 0   ┆ d[3,27,5]       │
│ 4,80,15      ┆ 1065.891604   ┆ 1   ┆ d[4,80,15]      │
│ …            ┆ …             ┆ …   ┆ …               │
│ 1054,331,225 ┆ -88.71552     ┆ 0   ┆ d[1054,331,225] │
│ 1055,280,360 ┆ 1.9233e-20    ┆ 0   ┆ d[1055,280,360] │
│ 1056,350,225 ┆ -110.733544   ┆ 0   ┆ d[1056,350,225] │
│ 1057,170,231 ┆ 0.0           ┆ 0   ┆ d[1057,170,231] │
│ 1058,247,318 ┆ -131.435155   ┆ 0   ┆ d[1058,247,318] │
└──────────────┴───────────────┴─────┴─────────────────┘


2025-06-23 15:00:18 B01BPC15 pipelines.dig_a_plan_d_model[128998] WARNING 
Master model did not converge: infeasible


In [5]:
dig_a_plan.edge_data.filter(~c("normal_open"))

nx_graph = nx.Graph()
_ = dig_a_plan.edge_data.filter(~c("normal_open")).with_columns(
    pl.struct(pl.all()).pipe(generate_nx_edge, nx_graph=nx_graph)
)
if nx.is_tree(nx_graph):
    print("The resulting graph is a tree")
    
else:
    print("The resulting graph is NOT a tree")
    


The resulting graph is a tree


In [6]:
list(nx.connected_components(G = nx_graph))

[{0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  62,
  63,
  64,
  65,
  68,
  70,
  71,
  72,
  73,
  75,
  77,
  80,
  81,
  82,
  83,
  84,
  85,
  87,
  88,
  89,
  90,
  91,
  92,
  98,
  99,
  100,
  101,
  102,
  103,
  105,
  106,
  107,
  108,
  109,
  110,
  111,
  112,
  114,
  115,
  116,
  117,
  118,
  119,
  120,
  121,
  122,
  123,
  124,
  125,
  126,
  127,
  128,
  129,
  130,
  131,
  132,
  133,
  134,
  135,
  136,
  137,
  138,
  139,
  140,
  141,
  142,
  143,
  144,
  145,
  146,
  147,
  148,
  149,
  150,
  151,
  152,
  153,
  155,
  157,
  158,
  159,
  160,
  161,
  162,
  163,
  164,
  165,
  166,
  167,
  168,
  169,
  170,
  171,
  172,
  173,
  174,
  176,
  177,
  178,
  179,
  180,
  181,
  1